### 라이브러리

In [1]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

### API 키

In [2]:
# 🔐 환경변수 로드
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

### PDF 로드 / 임베딩

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 📄 TXT 로드 및 분할
loader = TextLoader(r"C:\project_0728\project_12\park_geunho\KB_실버암_간편건강보험Plus.txt", encoding="utf-8")
docs_raw = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = text_splitter.split_documents(docs_raw)

# OpenAI 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              openai_api_key=OPENAI_API_KEY,
                              )

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever()


RuntimeError: Error loading C:\project_0728\project_12\park_geunho\KB_실버암_간편건강보험Plus.txt

In [ ]:
# Claude LLM
llm = ChatAnthropic(model="claude-opus-4-20250514",
                    temperature=0,
                    max_tokens=1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# ✅ LLM 단독 체인 구성 (출력 깔끔하게)
llm_only_chain = llm | StrOutputParser()

# 질문
# question = "고지의무를 위반한 경우 보험사는 어떤 조치를 취할 수 있나요?"                                  # 민석 질문
# question = "특정암 ii란 뭐야?"                                                                          # 1번 질문
# question = "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?"                # 2번 질문
question = "통합암진단비ii 는 보장범위가 어떻게 돼?"                                                      # 3번 질문
# question = "통합전이암진단비의 보장범위는 어떻게 돼?"                                                     # 4번 질문
# question = "보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?"      # 5번 질문
# question = "신재진단암진단비ii 에서 수술이란 뭐야?"                                                       # 6번 질문    
# question = "보험금청구 했는데 언제 지급되니?"                                                             # 7번 질문

# 1. RAG 기반 응답
rag_answer = rag_chain.invoke(question)

# 2. 순수 LLM 응답 (문서 없이 바로 Claude에게 질문)
llm_only_answer = llm_only_chain.invoke(question)

# 출력
print("🧾 질문:", question)
print("\n💬 [LLM 단독 응답]:")
print(llm_only_answer)
print("\n📚 [RAG 기반 응답]:")
print(rag_answer)



C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


🧾 질문: 통합암진단비ii 는 보장범위가 어떻게 돼?

💬 [LLM 단독 응답]:
통합암진단비II의 일반적인 보장범위를 안내드리겠습니다.

## 주요 보장 내용

**1. 일반암**
- 갑상선암, 기타피부암, 제자리암, 경계성종양을 제외한 모든 암
- 진단 시 가입금액의 100% 지급

**2. 갑상선암**
- 진단 시 가입금액의 20~30% 지급

**3. 기타피부암**
- 진단 시 가입금액의 10~20% 지급

**4. 제자리암(상피내암)**
- 진단 시 가입금액의 10~20% 지급

**5. 경계성종양**
- 진단 시 가입금액의 10~20% 지급

## 주요 특징

- **통합 보장**: 여러 종류의 암을 하나의 특약으로 보장
- **차등 지급**: 암의 종류에 따라 보험금을 차등 지급
- **1회 한정**: 각 암 종류별로 최초 1회만 지급

## 유의사항

- 보험사별로 세부 보장비율이 다를 수 있음
- 가입 후 90일간 면책기간 적용 (갑상선암 등 일부 제외)
- 정확한 보장내용은 가입하신 보험약관을 확인하시기 바랍니다

구체적인 상품명이나 보험사를 알려주시면 더 정확한 정보를 제공할 수 있습니다.

📚 [RAG 기반 응답]:
통합암진단비Ⅱ는 총 9개의 세부보장으로 구성되어 있으며, 특정소액암진단비Ⅱ, 생식기암 및 비뇨기관암진단비(자궁,전립선,방광제외), 소화기관암진단비, 혈액암진단비 등이 포함됩니다. 암보장개시일 이후 통합암Ⅱ(전이암포함)(유사암제외)로 진단확정된 경우, 암구분에 따라 각각 최초 1회의 진단에 한하여 보험금이 지급됩니다. 각 세부보장별로 보험가입금액의 50% 또는 100%가 지급되는 구조로 되어 있습니다.
